In [ ]:
!pip install opendatasets

In [ ]:
#Download dataset
import opendatasets as od
od.download("https://www.kaggle.com/datasets/saurabhbagchi/deepfake-image-detection")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: kjasmine19
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/saurabhbagchi/deepfake-image-detection


100%|██████████| 476M/476M [00:01<00:00, 448MB/s]


In [ ]:
#Imports
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split # Add this line
from transformers import AutoImageProcessor, AutoModelForImageClassification
import numpy as np
import pandas as pd
from tqdm import tqdm


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [ ]:
# Load dataset
class DeepfakeDataset(Dataset):
    def __init__(self, image_paths, labels, processor, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.processor = processor
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        inputs = self.processor(images=image, return_tensors="pt")
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        return inputs, label

In [ ]:
dataset_path = "/content/deepfake-image-detection/train-20250112T065955Z-001/train"
image_paths = []
labels = []

for label_folder in ['fake', 'real']:
    folder_path = os.path.join(dataset_path, label_folder)
    label = 1 if label_folder == 'fake' else 0
    for img_file in os.listdir(folder_path):
        if img_file.lower().endswith(('.jpg', '.png', '.jpeg')):
            image_paths.append(os.path.join(folder_path, img_file))
            labels.append(label)

df = pd.DataFrame({
    "image_path": image_paths,
    "label": labels
})


In [ ]:
# Limit for faster testing (100 real, 100 fake)
df = df.sample(frac=1, random_state=42)
df = df.groupby('label').head(100).reset_index(drop=True)

In [ ]:
df

,image_path,label
0,/content/deepfake-image-detection/train-202501...,1
1,/content/deepfake-image-detection/train-202501...,0
2,/content/deepfake-image-detection/train-202501...,1
3,/content/deepfake-image-detection/train-202501...,1
4,/content/deepfake-image-detection/train-202501...,0
...,...,...
195,/content/deepfake-image-detection/train-202501...,1
196,/content/deepfake-image-detection/train-202501...,1
197,/content/deepfake-image-detection/train-202501...,1
198,/content/deepfake-image-detection/train-202501...,1


In [ ]:
# Train-test split
train_df, test_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)

train_image_paths = train_df['image_path'].tolist()
train_labels = train_df['label'].tolist()
test_image_paths = test_df['image_path'].tolist()
test_labels = test_df['label'].tolist()

In [ ]:
def load_and_preprocess_images(image_paths, processor):
    images = []
    for img_path in tqdm(image_paths, desc="Loading Images"):
        img = Image.open(img_path).convert("RGB")
        inputs = processor(images=img, return_tensors="pt")
        images.append(inputs['pixel_values'][0])
    return torch.stack(images)

In [ ]:
model_names = [
    "google/vit-base-patch16-224",
    "microsoft/swin-base-patch4-window7-224",
    "microsoft/beit-base-patch16-224-pt22k-ft22k",
    "nvidia/mit-b0",
    "apple/mobilevit-small",
    "facebook/convnext-tiny-224",
    "google/efficientnet-b7"
]

results = []
predictions_all = []

In [ ]:
for i, model_name in enumerate(model_names):
    print(f"\nRunning Model {i+1}: {model_name}")

    try:
        # Load processor and model
        processor = AutoImageProcessor.from_pretrained(model_name)
        model = AutoModelForImageClassification.from_pretrained(model_name)

        # Modify the classifier head for binary classification
        if hasattr(model, 'classifier'):
            if isinstance(model.classifier, torch.nn.Linear):
                model.classifier = torch.nn.Linear(model.classifier.in_features, 2)
        elif hasattr(model, 'fc'):
            model.fc = torch.nn.Linear(model.fc.in_features, 2)
        elif hasattr(model, 'head'):
            model.head = torch.nn.Linear(model.head.in_features, 2)

        model = model.to(device)
        model.eval()

        # Create test dataset
        test_dataset = DeepfakeDataset(test_image_paths, test_labels, processor)
        test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

        preds = []
        with torch.no_grad():
            for inputs, _ in test_loader:
                inputs = {k: v.to(device) for k, v in inputs.items()}
                outputs = model(**inputs)
                logits = outputs.logits
                batch_preds = logits.argmax(dim=1).cpu().numpy()
                preds.extend(batch_preds)

        predictions_all.append(preds)

        # Calculate metrics
        acc = accuracy_score(test_labels, preds)
        prec = precision_score(test_labels, preds, average="macro", zero_division=0)
        rec = recall_score(test_labels, preds, average="macro", zero_division=0)
        f1 = f1_score(test_labels, preds, average="macro", zero_division=0)

        cm = confusion_matrix(test_labels, preds)
        if cm.shape == (2, 2):
            sens = cm[1, 1] / (cm[1, 0] + cm[1, 1]) if (cm[1, 0] + cm[1, 1]) > 0 else 0
            spec = cm[0, 0] / (cm[0, 1] + cm[0, 0]) if (cm[0, 1] + cm[0, 0]) > 0 else 0
        else:
            sens = spec = 0

        results.append([f"M{i+1}", sens, spec, prec, rec, f1, acc])
        print(f"Model {i+1} completed with accuracy: {acc:.4f}")

    except Exception as e:
        print(f"Error with model {model_name}: {str(e)}")
        continue



Running Model 1: google/vit-base-patch16-224
Model 1 completed with accuracy: 0.5667

Running Model 2: microsoft/swin-base-patch4-window7-224
Model 2 completed with accuracy: 0.4333

Running Model 3: microsoft/beit-base-patch16-224-pt22k-ft22k
Model 3 completed with accuracy: 0.5667

Running Model 4: nvidia/mit-b0
Model 4 completed with accuracy: 0.5000

Running Model 5: apple/mobilevit-small
Model 5 completed with accuracy: 0.5667

Running Model 6: facebook/convnext-tiny-224
Model 6 completed with accuracy: 0.4500

Running Model 7: google/efficientnet-b7
Model 7 completed with accuracy: 0.5000


In [ ]:
# Ensemble via majority voting
predictions_all = np.array(predictions_all)
ensemble_preds = np.round(predictions_all.mean(axis=0)).astype(int)

acc = accuracy_score(test_labels, ensemble_preds)
prec = precision_score(test_labels, ensemble_preds,average="macro")
rec = recall_score(test_labels, ensemble_preds,average="macro")
f1 = f1_score(test_labels, ensemble_preds,average="macro")
cm = confusion_matrix(test_labels, ensemble_preds)
sens = cm[1,1] / (cm[1,0] + cm[1,1]) if (cm[1,0] + cm[1,1]) > 0 else 0
spec = cm[0,0] / (cm[0,1] + cm[0,0]) if (cm[0,1] + cm[0,0]) > 0 else 0

results.append(["Ensemble", sens, spec, prec, rec, f1, acc])



In [ ]:
results_df = pd.DataFrame(results, columns=["Models", "Sensitivity", "Specificity", "Precision", "Recall", "F1", "Accuracy"])
print("\n Final Evaluation Table:\n")
print(results_df)



 Final Evaluation Table:

     Models  Sensitivity  Specificity  Precision    Recall        F1  Accuracy
0        M1     0.433333     0.700000   0.571770  0.566667  0.558824  0.566667
1        M2     0.200000     0.666667   0.414773  0.433333  0.400705  0.433333
2        M3     0.833333     0.300000   0.593168  0.566667  0.533493  0.566667
3        M4     0.500000     0.500000   0.500000  0.500000  0.500000  0.500000
4        M5     0.700000     0.433333   0.571770  0.566667  0.558824  0.566667
5        M6     0.533333     0.366667   0.448571  0.450000  0.446154  0.450000
6        M7     0.600000     0.400000   0.500000  0.500000  0.494949  0.500000
7  Ensemble     0.566667     0.500000   0.533482  0.533333  0.532814  0.533333
